# **Year 3 project - Optimising the stability of a catalyst**

**Data Loading + imports**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from matminer.featurizers.conversions import StrToComposition
from pymatgen.ext.matproj import MPRester
from sklearn.linear_model import LinearRegression
from IPython.display import display
initial_df = pd.read_csv(r'Python import.csv')

**Data exploration**

In [ ]:
def dataset_info(dataset):
    print(dataset.columns)
    print(dataset.describe())
    print(dataset.info())
    display(dataset)

**Data cleaning functions**

In [ ]:
#This function will replace '-' values for zeroes. Also replace Nan values with zeroes
def parse_Nan(dataset):
    parse_datatset = dataset.replace('-',0) 
    parse_datatset = parse_datatset.fillna(0)
    return parse_datatset

In [ ]:
#filter columns that are required for us. Also convert some object columns to floats
def column_cleaning(dataset):
    columns_to_keep = ["Catalyst Materials","Family","ΔGOH*","ΔGO*","ΔGOOH*","Overpotential at xx (nearby) current density vs RHE, V"]
    updated_data = dataset.loc[:,columns_to_keep]
    columns_to_int = ["ΔGOH*","ΔGO*","ΔGOOH*","Overpotential at xx (nearby) current density vs RHE, V"]
    updated_data[columns_to_int] = updated_data[columns_to_int].astype(float)
    return updated_data

**Launching cleaning functions on datasets**

In [ ]:
#Launching functions

initial_df = parse_Nan(initial_df)
updated_data = column_cleaning(initial_df)
updated_data['ΔGO*-ΔGOH*'] = updated_data['ΔGO*'] - updated_data['ΔGOH*']
#updated_data['ΔGOOH*-ΔGOH*'] = updated_data['ΔGOOH*'] - updated_data['ΔGOH*']
#Slicing only data with values for required columns, then plotting dataset
updated_data = updated_data.loc[updated_data["ΔGO*-ΔGOH*"] != 0.0 ]
dataset_info(updated_data)
print(updated_data.describe())

**Element composition featurisation**

In [ ]:
#Creating a column for element composition
str_comp = StrToComposition(target_col_id='composition')
updated_data = str_comp.featurize_dataframe(updated_data, col_id='Catalyst Materials')

**Accessing material ID from material project + other properties**

In [ ]:
with MPRester('CmGarHKLtPCjVpEkivPDYkhRJKJiB8A7') as mpr:
    results = mpr.summary.search(formula='**O3', fields=["material_id", "formula_pretty",'band_gap','chemsys'])
    data_list = [(result.material_id, result.formula_pretty, result.band_gap,result.chemsys) for result in results]
    #print(data_list)

In [ ]:
#Sorting chemical compositions in formulas alphabetically

def alph_order(composition):
    new_formula = []
    for i in composition:
        first= []
        second= []
        temp = list(i)
        capital_pos=[]
        for x in range(0, len(temp)):
            if temp[x].isupper() == True:
                capital_pos.append(x)
        for y in range(capital_pos[0], capital_pos[1]):
                first.append(temp[y])
        for z in range(capital_pos[1], len(temp)):
                second.append(temp[z])
        first = ''.join(str(v) for v in first)
        second = ''.join(str(v) for v in second)
        compound = ''.join(str(v) for v in first)
        compound = second+first+"O3"
        new_formula.append(compound)
    return new_formula

#This function is to delete the extra O3 placed in the formulas

def remove_O3(composition):
    return [i.replace("O3", "") for i in composition]

def add_o3(composition):
    return [i + "O3" for i in composition]



**Sorting list to a dictionary**

In [ ]:
formula = []
material_id = []
band_gap = []
chemsys = []
for i in data_list:
    material_id.append(i[0])
    formula.append(i[1])
    band_gap.append(i[2])
    chemsys.append(i[3])
#Creating dictionary
material_proj_data = dict((z[0], list(z[1:])) for z in zip(formula, material_id, band_gap, chemsys))
rearranged_formula = alph_order(formula)
rearranged_formula = remove_O3(rearranged_formula)
rearranged_formula = add_o3(rearranged_formula)
#Creating dictionary for re-arranged formulae
material_proj_data_rearranged = dict((z[0], list(z[1:])) for z in zip(rearranged_formula, material_id, band_gap, chemsys))
material_proj_data.update(material_proj_data_rearranged)

**Dictionary to pandas dataframe + merge with original dataset**

In [ ]:
material_proj_df = pd.DataFrame.from_dict(material_proj_data, orient= 'index')
material_proj_df = material_proj_df.reset_index()
material_proj_df.columns = ["Catalyst Materials", "material id", "band gap","chemsys"]
updated_data = updated_data.merge(material_proj_df, on ='Catalyst Materials', how= 'left')

**Dataset exploration after importing data**

In [ ]:
#dataset_info(updated_data)

**Preparing first graph**

In [ ]:
def plot_scatter_reaction_path(data):
    x_variables = ['ΔGO*-ΔGOH*','ΔGO*','ΔGOH*']
    for i in x_variables:
        plt.scatter(data[i],data["Overpotential at xx (nearby) current density vs RHE, V"])
        plt.xlabel(i)
        plt.ylabel('Overpotential at xx (nearby) current density vs RHE, V')
        if i=='ΔGO*-ΔGOH*':
            plt.savefig('Scatter_plots\Overpotential against reaction step (GO-GOH).png',bbox_inches='tight')
        elif i=='ΔGO*':
            plt.savefig('Scatter_plots\Overpotential against reaction step (GO).png',bbox_inches='tight')
        else:
            plt.savefig('Scatter_plots\Overpotential against reaction step (GOH).png',bbox_inches='tight')
        plt.show()

**Clustering**

In [ ]:
#Creating an elbow plot to determine how many cluster will be required for model
def elbow_plot(elbow_data):
    k_rng = range(1,10)
    sse = []
    for k in k_rng:
        km = KMeans(n_clusters= k)
        km.fit(elbow_data[['ΔGO*-ΔGOH*','Overpotential at xx (nearby) current density vs RHE, V']])
        sse.append(km.inertia_)
    plt.xlabel('k')
    plt.ylabel('Sum of squared error')
    plt.plot(k_rng,sse)
    plt.savefig('Scatter_plots\Elbow plot .png',bbox_inches='tight')
    plt.show()

In [ ]:
#Creating cluster plot using K-means method
def cluster_df(cluster_df,n_clusters):
    color = ['black','green','purple','red','blue']
    for k in range(0,n_clusters):
        data = cluster_df[cluster_df["cluster"]==k]
        plt.scatter(data['ΔGO*-ΔGOH*'],data["Overpotential at xx (nearby) current density vs RHE, V"],c=color[k])
        # Save datasets
        if k==0:
            data.to_csv('Cluster_csv_files\Cluster_0.csv')
        elif k==1:
                data.to_csv('Cluster_csv_files\Cluster_1.csv')
        elif k==2:
            data.to_csv('Cluster_csv_files\Cluster_2.csv')
        elif k==3:
            data.to_csv('Cluster_csv_files\Cluster_3.csv')
        else:
            data.to_csv('Cluster_csv_files\Cluster_4.csv')
    plt.xlabel('ΔGO*-ΔGOH*')
    plt.ylabel('Overpotential at xx (nearby) current density vs RHE, V')
    plt.savefig('Scatter_plots\Overpotential against reaction step (with regions).png',bbox_inches='tight')
    plt.show()
    for k in range(0,n_clusters):
        data = cluster_df[cluster_df["cluster"]==k]
        plt.scatter(data['ΔGOH*'],data['ΔGOOH*'],c=color[k])
    plt.xlabel('ΔGOH*')
    plt.ylabel('ΔGOOH*')
    plt.savefig('Scatter_plots\GOH against GOOH .png',bbox_inches='tight')
    plt.show()
    for k in range(0,n_clusters):
        data = cluster_df[cluster_df["cluster"]==k]
        plt.scatter(data['ΔGO*'],data['ΔGOH*'],c=color[k])
    plt.xlabel('ΔGO*')
    plt.ylabel('ΔGOH*')
    plt.savefig('Scatter_plots\GO against GOH .png',bbox_inches='tight')
    plt.show()
    

In [ ]:
# Code to form linear regression model of GOOH* against GOH*
def linear_regression_model(dataset,x,y):
    reg = LinearRegression()
    reg.fit(dataset[[x]], dataset[[y]])
    slope = reg.coef_[0][0]
    print(slope)

In [ ]:
#Placing functions down
plot_scatter_reaction_path(updated_data)
elbow_plot(updated_data)
km = KMeans(n_clusters= 5)
y_predicted = km.fit_predict(updated_data[['ΔGO*-ΔGOH*','Overpotential at xx (nearby) current density vs RHE, V']])
updated_data['cluster'] = y_predicted
cluster_df(updated_data,5)
linear_regression_model(updated_data,'ΔGOH*','ΔGOOH*')
linear_regression_model(updated_data,'ΔGO*','ΔGOH*')

**Data exploration of optimum cluster**

In [ ]:
#Function to determine proprtion of different catalyst materials in each dataset
def printing_stats_1(df_1,df_2,df_3):
    df_names = ['optimum catalysts','filtered initial','initial']
    data = [df_1, df_2 ,df_3]
    for i in range(0,3):
        print('The number of families in the', df_names[i], 'dataframe + normalized is:')
        print(data[i]['Family'].value_counts())
        print(data[i]['Family'].value_counts(normalize= True))

In [ ]:
#Summary statistics on family data on optimum catalysts, catalyst filtered down to our data and the whole initial DF
optimum_catalyst = updated_data[updated_data['Overpotential at xx (nearby) current density vs RHE, V']<1.10]
#printing_stats_1(optimum_catalyst,updated_data,initial_df)